# `散佈圖 - 如何找出高CP值產品？ 網路輿情與產品定位分析實戰案例`
### 作者：徐子皓、張友志
***

## 引入相關檔案

### 引入套件包

In [1]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.offline as py
import pandas as pd
import numpy as np
import jieba
jieba.set_dictionary('dict.txt.big.txt') # 讀取結巴字典

### 引入輿情資料集

In [2]:
KoreaDrama=pd.read_csv('CH3-4_影劇輿情資料.csv', encoding='utf-8-sig')
KoreaDrama.head()

,標題,時間,內容
0,[LIVE] 2018 SBS 演技大賞,20181231,"<span class=""b1 hl"">2018 SBS 演技大賞</span>\r\n日 ..."
1,[LIVE] 2018 KBS 演技大賞,20181231,2018 KBS演技大賞\r\n\r\n 直播時間：2018/12/31 19:55\...
2,[心得] 值得一看的就算死也喜歡(雷),20181231,繼金科長後~終於又看到一部讓人忍不住一集接著一集看下去的戲 說這部像金科長，但又\r\n多了...
3,Re: [LIVE] tvN 阿爾罕布拉宮的回憶 EP10,20181231,以下皆爆雷，請慎入！\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n...
4,[心得]金材昱、徐睿知：另有他路(雷),20181231,"<a href=""https://i.imgur.com/ZFcWKLL.jpg"" rel=..."


## 進行關鍵字統計

### 關注影劇名稱

In [3]:
movie = ['成為王的男人','皇后的品格','赤月青日','神的測驗',
        '死之詠讚','王牌大律師','Priest驅魔者','加油吧威基基',
        '皮諾丘','魔女寶鑑','好運羅曼史','購物王路易','七次初吻',
        '男朋友','請回答1997','來自星星的你']
# 新增到結巴字典中
for i in movie:
    jieba.add_word(i)

Building prefix dict from c:\Users\user\Desktop\Nash校稿\dict.txt.big.txt ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.u5cdeee7902449e994d3259f463c04a8c.cache
Loading model cost 0.443 seconds.
Prefix dict has been built successfully.


### 無意義字元列表，可以自行新增

In [4]:
removeword = ['span','class','f3','https','imgur','h1','_   blank','href','rel',
              'nofollow','target','cdn','cgi','b4','jpg','hl','b1','f5','f4',
              'goo.gl','f2','email','map','f1','f6','__cf___','data','bbs',
              'html','cf','f0','b2','b3','b5','b6','原文內容','原文連結','作者'
              '標題','時間','看板','<','>','，','。','？','—','閒聊','・','/',
              ' ','=','\"','\n', '\r','」','「','！','[',']','：','‧','╦','╔','╗','║'
              ,'╠','╬','╬',':','╰','╩','╯','╭','╮','│','╪','─','《','》','_'
              ,'.','、','（','）','　','*','※','~','○','”','“','～','@','＋']

### 將無意義字元移除

In [5]:
'好棒棒'.replace('好', '') # 概念

'棒棒'

In [6]:
# 將標題與內容合併，移除無意義字元
KoreaDrama['標題與內容'] = KoreaDrama['標題'] +  KoreaDrama['內容']
for word in removeword:
    KoreaDrama['標題與內容'] = KoreaDrama['標題與內容'].str.replace(word,'')
KoreaDrama.head()['標題與內容']

0    LIVE2018SBS演技大賞2018SBS演技大賞日期20181231時間2055主持人申...
1    LIVE2018KBS演技大賞2018KBS演技大賞直播201812311955直播網址1a...
2    心得值得一看的就算死也喜歡(雷)繼金科長後終於又看到一部讓人忍不住一集接著一集看下去的戲說這...
3    ReLIVEtvN阿爾罕布拉宮的回憶EP10以下皆爆雷請慎入E10喪屍席捲而來昨晚看此集的情...
4    心得金材昱徐睿知另有他路(雷)aicomZFcWKLLblankicomZFcWKLLa雖然...
Name: 標題與內容, dtype: object

### 將「標題與內容」全部併在一起

In [7]:
theSTR = KoreaDrama['標題與內容'].tolist() 
theSTR = ''.join(theSTR )
#theSTR

### 進行切詞

In [8]:
words = jieba.lcut(theSTR)
words[:20] # 前20個切詞成果

['LIVE2018SBS',
 '演技大賞',
 '2018SBS',
 '演技大賞',
 '日期',
 '20181231',
 '時間',
 '2055',
 '主持人',
 '申東燁',
 '申惠善',
 '李帝勳',
 '網頁',
 'a5izEtoblank5izEtoa',
 '(',
 '感謝',
 'watase124',
 '提供',
 ')',
 '直播']

### 計算關注影劇名稱出現頻率

In [9]:
words.count('來自星星的你') # 概念

10

In [10]:
# 實際計算
# 計算關注影劇名稱出現頻率
mv_voice= []
for j in movie:
    mv_voice.append(words.count(j))
# 計算平均頻率
avg=np.mean(mv_voice)
avg

17.1875

### 記錄豆瓣評分，計算平均分數

In [11]:
bean = [6.9, 7.5, 8.6, 7.8, 8.5, 9.3, 6.8, 8.6, 8.2, 5.9, 6.7, 7.2, 5.8, 7.0, 9.0, 8.3]
# 平均評分
score_avg = np.mean(bean)
score_avg

7.63125

### 透過迴圈，逐一取得不同影劇的相關資料

In [12]:
voice_list =[] # 用以儲存聲量
bean_list = [] # 用以儲存豆瓣評分 
axe_list = [] # 用以儲存象限
for i in range(len(bean)):
    if bean[i]>score_avg and mv_voice[i] >avg:#1_第一象限
        axe = '1_第一象限'
        
    elif bean[i]>score_avg and mv_voice[i] <= avg:#4_第四象限
        axe = '4_第四象限'
        
    elif bean[i]<=score_avg and mv_voice[i] > avg:#2_第二象限
        axe = '2_第二象限'
        
    else:#2_第三象限
        axe = '3_第三象限'
            
    voice_list.append(mv_voice[i])
    bean_list.append(bean[i])
    axe_list.append(axe)

### 將影劇相關資料製作成資料集，並匯出

In [13]:
final1 = pd.DataFrame(zip(voice_list, bean_list, axe_list, movie), columns=['聲量', '評分', '象限', '劇名'])
final1 = final1.sort_values('象限')
final1.to_csv('CH3-4產出成果_評分與聲量表.csv', encoding = 'utf-8-sig')
final1

,聲量,評分,象限,劇名
2,27,8.6,1_第一象限,赤月青日
3,26,7.8,1_第一象限,神的測驗
4,39,8.5,1_第一象限,死之詠讚
8,20,8.2,1_第一象限,皮諾丘
1,62,7.5,2_第二象限,皇后的品格
13,52,7.0,2_第二象限,男朋友
0,4,6.9,3_第三象限,成為王的男人
6,9,6.8,3_第三象限,Priest驅魔者
9,14,5.9,3_第三象限,魔女寶鑑
10,0,6.7,3_第三象限,好運羅曼史


## 資料視覺化

In [14]:
fig = px.scatter(final1, x="評分", y="聲量", color="象限", hover_data=["劇名"])
fig.update_layout(
        title = '評分與聲量圖',
        shapes=[
        # 設定X軸
        dict({
            'type': 'line',
            'x0': score_avg,
            'y0': -10,
            'x1': score_avg,
            'y1': final1['聲量'].max()*1.1,
            'line': {
                'color': '#FF00FF',
                'width': 2}}),
        # 設定Y軸
        dict({
            'type': 'line',
            'x0': 0,
            'y0': avg,
            'x1': final1['評分'].max()*1.1,
            'y1': avg,
            'line': {
                'color': '#FF00FF',
                'width': 2}})])
fig.show()

### 將成果另存新檔

In [15]:
# 圖檔
fig.write_image('CH3-4產出成果_評分與聲量圖.png')
# 網頁檔
py.plot(fig, filename='CH3-4產出成果_評分與聲量圖', auto_open=True)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `CH3-4產出成果_評分與聲量圖` didn't end with .html. Adding .html to the end of your file.



'CH3-4產出成果_評分與聲量圖.html'

## 加入影劇購買成本做為考量，再進行製圖

### 引入影劇購買成本表

In [16]:
drama_cost = pd.read_csv('CH3-4_影劇購買成本表.csv', encoding = 'utf-8-sig')
drama_cost

,劇名,cost
0,成為王的男人,190000
1,皇后的品格,2000000
2,赤月青日,760000
3,神的測驗,450000
4,死之詠讚,1950000
5,王牌大律師,930000
6,Priest驅魔者,480000
7,加油吧威基基,860000
8,皮諾丘,820000
9,魔女寶鑑,590000


### 計算平均購買成本

In [17]:
avg_cost = drama_cost['cost'].mean()
avg_cost

895625.0

### 透過迴圈，逐一取得不同影劇的相關資料

In [18]:
voice_list =[] # 用以儲存聲量
cost_list = [] # 用以儲存成本
axe_list = [] # 用以儲存象限
for i in range(len(drama_cost['cost'])):
    if drama_cost['cost'][i]>avg_cost and mv_voice[i] >avg:#1_第一象限
        axe = '1_第一象限'
        
    elif drama_cost['cost'][i]>avg_cost and mv_voice[i] <= avg:#4_第四象限
        axe = '4_第四象限'
        
    elif drama_cost['cost'][i]<=avg_cost and mv_voice[i] > avg:#3_第三象限
        axe = '2_第二象限'
        
    else:#2_第二象限
        axe = '3_第三象限'

    voice_list.append(mv_voice[i])
    cost_list.append(drama_cost['cost'][i])
    axe_list.append(axe)

### 將影劇相關資料製作成資料集，並匯出

In [19]:
final2 = pd.DataFrame(zip(voice_list, cost_list, axe_list, movie), columns=['聲量', '成本', '象限', '劇名'])
final2 = final2.sort_values('象限')
final2.to_csv('CH3-4產出成果_成本效益評估分析表.csv', encoding = 'utf-8-sig')
final2

,聲量,成本,象限,劇名
1,62,2000000,1_第一象限,皇后的品格
4,39,1950000,1_第一象限,死之詠讚
2,27,760000,2_第二象限,赤月青日
3,26,450000,2_第二象限,神的測驗
8,20,820000,2_第二象限,皮諾丘
13,52,700000,2_第二象限,男朋友
0,4,190000,3_第三象限,成為王的男人
6,9,480000,3_第三象限,Priest驅魔者
7,12,860000,3_第三象限,加油吧威基基
9,14,590000,3_第三象限,魔女寶鑑


## 資料視覺化

In [20]:
fig = px.scatter(final2, x = '成本', y = '聲量', size = '成本', color = '象限',  hover_data=['劇名'])
fig.update_layout(
        title = '成本效益評估分析',
        shapes=[
        # 設定X軸
        dict({
            'type': 'line',
            'x0': avg_cost,
            'y0': -10,
            'x1': avg_cost,
            'y1': final2['聲量'].max()*1.2,
            'line': {
                'color': '#FF00FF',
                'width': 2}}),
        # 設定Y軸
        dict({
            'type': 'line',
            'x0': 0,
            'y0': avg,
            'x1': final2['成本'].max()*1.1,
            'y1': avg,
            'line': {
                'color': '#FF00FF',
                'width': 2}})])
fig.show()

### 將成果另存新檔

In [21]:
# 圖檔
fig.write_image('CH3-4產出成果_成本效益評估分析圖.png')
# 網頁檔
py.plot(fig, filename='CH3-4產出成果_成本效益評估分析圖', auto_open=True)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `CH3-4產出成果_成本效益評估分析圖` didn't end with .html. Adding .html to the end of your file.



'CH3-4產出成果_成本效益評估分析圖.html'